In [33]:
# Installations

%pip install --upgrade pip -q
%pip install python-dotenv -q

%pip install langchain langchain-core -q

%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
from dotenv import dotenv_values

environment_secrets = dotenv_values("../.env")

In [35]:
from langsmith import Client

langsmith_client = Client(api_key=environment_secrets["LANGSMITH_API_KEY"])

In [38]:
from functools import reduce

from pandas import read_csv
from datasets import Dataset

from rag_qa_chain import build_chain
from typing import Any

from pprint import pprint

from json import dump, load

def main():
	openai_api_key = environment_secrets["OPENAI_API_KEY"] 
	assert openai_api_key != None

	test_set = read_csv("./test_set.csv")
	test_questions: list[str] = test_set["question"].values.tolist()
	test_ground_truths: list[str] = test_set["ground_truth"].values.tolist()

	rag_qa_chain = build_chain(openai_api_key)

	def parse_answers_and_context(answers_contexts: dict[str, list[str]], question: str):
		response = rag_qa_chain.invoke({"query": question})
		answers_contexts["answer"].append(response["response"].content)
		answers_contexts["contexts"].append([context.page_content for context in response["context"]])
		return answers_contexts

	# def transform(mapping: dict[str, list[Any]]) -> list[dict[str, Any]]:
	# 	column_names = mapping.keys()

	# 	return [
	# 		{column_name: value for column_name, value in zip(column_names, row)} for 
	# 		row in 
	# 		zip(*(mapping[column_name] for column_name in column_names))
	# 	]

	# with open("./response_set.json") as response_set_file:
	# 	response_set: dict[str, Any] = load(response_set_file)
    
	# response_set["contexts"] = [",".join(contexts) for contexts in response_set["contexts"]]

	dataset = Dataset.from_dict({
		"question" : test_questions,
		**reduce(
			parse_answers_and_context,
			test_questions,
			{
				"answer": [],
				"contexts": []
			}
		),
		"ground_truth": test_ground_truths
	})

	dataset.to_csv("response_set.csv")


In [39]:

from langchain_core.tracers.context import tracing_v2_enabled
from uuid import uuid4

if __name__ == "__main__":
	with tracing_v2_enabled(project_name=f"Suds Ai - LangGraph - {uuid4().hex[0:8]}", client=langsmith_client):
		main()

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 13.09ba/s]
